In [8]:
# API KEY
from dotenv import load_dotenv
load_dotenv()

# LangSmith
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging
logging.langsmith("fear-greedy")

LangSmith 추적을 시작합니다.
[프로젝트명]
fear-greedy


In [2]:
url = "https://api.alternative.me/fng/?date_format=kr&limit="

In [28]:
import requests
import json

def get_fear():
    # 30일로 설정
    _url = url + "30"
    res = requests.request("GET", _url)

    parsed = json.loads(res.text)
    data = parsed["data"]

    # 데이터의 value 값만 리스트로 추출
    values = [int(item['value']) for item in data]

    return values

In [29]:
print(get_fear())

[69, 71, 70, 72, 73, 72, 73, 71, 73, 65, 48, 50, 49, 32, 39, 49, 49, 50, 50, 49, 41, 37, 42, 50, 61, 63, 64, 61, 50, 59]


In [35]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from typing import Literal, TypedDict

# 1. model
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

# 2. template
template = """당신은 투자 분석 전문가 입니다. 
    현재 공포 탐욕 지수의 최근 30일 지수의 정보는 다음과 같습니다: {values}

    분석 결과를 다음 JSON 형식으로 제공해주세요:
    {{
        "summary": "시장 분석 및 예측 내용을 상세히 서술",
        "decision": "BUY 또는 SELL 중 하나로만 작성"
    }}

    주의사항:
    - summary는 현재 시장 상황과 향후 전망을 포함해야 합니다
    - decision은 반드시 BUY 또는 SELL 중 하나여야 합니다
    """

prompt_template = PromptTemplate.from_template(template)

# 3. outputParser
class Analysis(TypedDict):
    summary: str
    decision: Literal["BUY", "SELL"]

output_parser = JsonOutputParser(pydantic_object=Analysis)

prompt_template

PromptTemplate(input_variables=['values'], template='당신은 투자 전문가 입니다. \n    현재 공포 탐욕 지수의 최근 30일 지수의 정보는 다음과 같습니다. {values}\n\n    분석 결과를 다음 JSON 형식으로 제공해주세요:\n    {{\n        "summary": "시장 분석 및 예측 내용을 상세히 서술",\n        "decision": "BUY 또는 SELL 중 하나로만 작성"\n    }}\n\n    주의사항:\n    - summary는 현재 시장 상황과 향후 전망을 포함해야 합니다\n    - decision은 반드시 BUY 또는 SELL 중 하나여야 합니다\n    ')

In [42]:
# chain
chain = prompt_template | model | output_parser
chain

values = get_fear()
result = chain.invoke({"values": values})
print(result)
print('summary: ', result['summary'])
print('decision: ', result['decision'])

{'summary': '최근 30일간의 공포 탐욕 지수를 분석한 결과, 최근에는 상승세를 보이고 있었지만, 이후 하락세를 보이며 안정화되고 있는 모습을 보입니다. 현재 시장은 상당히 변동성이 높은 상황이며, 투자자들의 불안감이 높아지고 있는 것으로 판단됩니다. 미래 전망은 불확실하며, 추가적인 경제 지표와 이벤트에 따라 시장이 크게 변동할 수 있습니다.', 'decision': 'SELL'}
summary:  최근 30일간의 공포 탐욕 지수를 분석한 결과, 최근에는 상승세를 보이고 있었지만, 이후 하락세를 보이며 안정화되고 있는 모습을 보입니다. 현재 시장은 상당히 변동성이 높은 상황이며, 투자자들의 불안감이 높아지고 있는 것으로 판단됩니다. 미래 전망은 불확실하며, 추가적인 경제 지표와 이벤트에 따라 시장이 크게 변동할 수 있습니다.
decision:  SELL
